# Modelling 

In [19]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium 
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor

In [20]:
# Create a spark session with increased memory allocation
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "8g")  # Set the driver memory to 8GB
    .config("spark.executor.memory", "8g")  # Set the executor memory to 8GB
    .getOrCreate()
)

## Dataset

In [21]:
# import the data
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [22]:
# Load the data
df = spark.read.parquet("../data/curated/tlc_data/first_cleaned.parquet")

In [23]:
# Drop missing values
df = df.dropna()

In [24]:
# compare the % of rows before and after dropping missing values
print(f"Number of rows before dropping missing values: {df.count()}")
print(f"Number of rows after dropping missing values: {df.dropna().count()}")
print(f"Percentage of rows removed: {100*(1 - df.dropna().count()/df.count()):.2f}%")

Number of rows before dropping missing values: 38461
Number of rows after dropping missing values: 38461
Percentage of rows removed: 0.00%


## Feature selection

In [25]:
# List of columns to drop
columns_to_drop = [
    'VendorID', 'passenger_count', 'RatecodeID', 'store_and_fwd_flag',
    'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 
    'tolls_amount', 'improvement_surcharge', 'total_amount', 
    'congestion_surcharge', 'ehail_fee', 'DOLocationID', 'DOBorough','dropoff_hour',
    'trip_distance', 'trip_duration'
]

# Dropping all specified columns at once in PySpark
df = df.drop(*columns_to_drop) 

In [26]:
# check the schema
df.printSchema()

root
 |-- datetime: timestamp (nullable = true)
 |-- PUBorough: string (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- hourly_trip_count: long (nullable = true)
 |-- Number of Events: long (nullable = true)
 |-- CIG: double (nullable = true)
 |-- WND: double (nullable = true)
 |-- VIS: double (nullable = true)
 |-- TMP: double (nullable = true)
 |-- DEW: double (nullable = true)
 |-- SLP: double (nullable = true)



## Data split

In [27]:
# Splitting the data into training (60%), validation (20%), and test (20%)
train_df, validation_df, test_df = df.randomSplit([0.6, 0.2, 0.2], seed=42)

In [28]:
train_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in train_df.columns]).show()

+--------+---------+-----------+-----------+-----------------+----------------+---+---+---+---+---+---+
|datetime|PUBorough|pickup_date|pickup_hour|hourly_trip_count|Number of Events|CIG|WND|VIS|TMP|DEW|SLP|
+--------+---------+-----------+-----------+-----------------+----------------+---+---+---+---+---+---+
|       0|        0|          0|          0|                0|               0|  0|  0|  0|  0|  0|  0|
+--------+---------+-----------+-----------+-----------------+----------------+---+---+---+---+---+---+



## Linear regression

In [29]:
# List of categorical and numerical columns
categorical_columns = ['PUBorough'] 
numerical_columns = [
    'pickup_hour', 
    'CIG', 'WND', 'VIS', 'TMP', 'DEW', 'SLP', 'Number of Events'
]

In [30]:
# Indexing and Encoding categorical columns
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categorical_columns]
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_ohe") for col in categorical_columns]

In [31]:
# Assemble the feature vector
assembler = VectorAssembler(
    inputCols=[
        'pickup_hour', 
        'CIG', 'WND', 'VIS', 'TMP', 'DEW', 'SLP', 'Number of Events',
        'PUBorough_ohe'
    ], 
    outputCol="features"
)


In [32]:
# Standardization
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")


In [33]:
# Linear Regression
lr = LinearRegression(featuresCol="scaled_features", labelCol="hourly_trip_count")

In [34]:
# Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler, lr])

In [35]:
# Fit the model on the training set
model = pipeline.fit(train_df)

24/08/29 22:22:28 WARN Instrumentation: [9620d3ba] regParam is zero, which might cause numerical instability and overfitting.
24/08/29 22:22:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/29 22:22:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
24/08/29 22:22:28 WARN Instrumentation: [9620d3ba] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [36]:
# Mean hourly trip count
mean_hourly_trip_count = train_df.agg(F.mean("hourly_trip_count")).head()[0]
print(f"Mean hourly trip count: {mean_hourly_trip_count}")

Mean hourly trip count: 818.7936528441631


In [37]:
# Evaluate the model on the validation set
validation_predictions = model.transform(validation_df)
evaluator = RegressionEvaluator(labelCol="hourly_trip_count", predictionCol="prediction", metricName="rmse")
validation_rmse = evaluator.evaluate(validation_predictions)
print(f"Root Mean Squared Error (RMSE) on validation data = {validation_rmse}")

Root Mean Squared Error (RMSE) on validation data = 889.5794710544728


In [38]:
# If satisfied with validation performance, evaluate on the test set
test_predictions = model.transform(test_df)
test_rmse = evaluator.evaluate(test_predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {test_rmse}")

Root Mean Squared Error (RMSE) on test data = 892.2997045096371


compare varaince of mean hourly count 

## Gradient Boosting


In [39]:
# Gradient Boosted Trees Regressor
gbt = GBTRegressor(featuresCol="scaled_features", labelCol="hourly_trip_count", maxIter=100, maxDepth=5)

# Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler, gbt])

# Fit the model on the training set
model = pipeline.fit(train_df)


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=47987Kb max_used=47999Kb free=83084Kb
 bounds [0x00000001089e8000, 0x000000010b928000, 0x00000001109e8000]
 total_blobs=16984 nmethods=15040 adapters=1855
 compilation: disabled (not enough contiguous free space left)


In [40]:
# Evaluate the model on the validation set
validation_predictions = model.transform(validation_df)
evaluator = RegressionEvaluator(labelCol="hourly_trip_count", predictionCol="prediction", metricName="rmse")
validation_rmse = evaluator.evaluate(validation_predictions)
print(f"Root Mean Squared Error (RMSE) on validation data = {validation_rmse}")

# If satisfied with validation performance, evaluate on the test set
test_predictions = model.transform(test_df)
test_rmse = evaluator.evaluate(test_predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {test_rmse}")

Root Mean Squared Error (RMSE) on validation data = 495.8899650269797
Root Mean Squared Error (RMSE) on test data = 485.31324130901976


## Interpret the model

### Identify which features are most important in making predictions

In [41]:
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from sklearn.inspection import plot_partial_dependence
from sklearn.ensemble import RandomForestRegressor as SklearnRandomForestRegressor
import matplotlib.pyplot as plt

# Assuming your DataFrame is named clean_df

# List of features to be used for prediction
feature_columns = ['passenger_count', 'trip_distance', 'trip_duration', 'RatecodeID', 
                   'hourly_trip_count', 'daily_trip_count', 'CIG', 'WND', 'VIS', 
                   'TMP', 'DEW', 'SLP', 'Number of Events', 'Day of Week']

# Assemble features into a vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

# Define the Random Forest model
rf = RandomForestRegressor(featuresCol='features', labelCol='hourly_trip_count')

# Create a pipeline
pipeline = Pipeline(stages=[assembler, rf])

# Split the data into training and test sets
train_data, test_data = clean_df.randomSplit([0.8, 0.2], seed=42)

# Train the model
model = pipeline.fit(train_data)

# Extract the feature and label columns to Pandas
pandas_df = test_data.select(feature_columns + ['hourly_trip_count']).toPandas()

# Train a RandomForestRegressor model using sklearn (since PDP is not available in PySpark)
X = pandas_df[feature_columns]
y = pandas_df['hourly_trip_count']

sklearn_rf = SklearnRandomForestRegressor(n_estimators=100)
sklearn_rf.fit(X, y)

# Plot Partial Dependence Plots for a few features
features_to_plot = ['trip_distance', 'trip_duration', 'Number of Events']  # Select features you want to plot

plot_partial_dependence(sklearn_rf, X, features_to_plot, grid_resolution=50)

plt.show()


ImportError: cannot import name 'plot_partial_dependence' from 'sklearn.inspection' (/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/inspection/__init__.py)